## Computing Artifact
### Author: Brian KYANJO
#### Supervisor: Prof. Donna Calhoun
###### Course: Computing PHD (Computational Math Science and Engineering)
This is a notebook where all test cases are tested. The mathematics and further details behind the implement solves is well discussed in the pdf file attached named brian_kyanjo_synthesis_duplicate.pdf.  

In [1]:
%matplotlib notebook
%pylab
%reload_ext autoreload
%autoreload 2

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Problem test
case = 8 # 0 = left going shock
         # 1 = right going shock
         # 2 = right going rarefaction
         # 3 = left going rarefaction
         # 4 = dambreak
         # 5 = Allrarefaction
         # 6 = Allshock
         # 7 = left dry state
         # 8 = middle dry state
         # 9 = right dry state

itype = 1 # 0 = presence  dry states (All exact solver)
          # 1 = no dry states (Only a selected approximate solver is compared to the exact)
          # 2 = no dry states (All solvers)
    
mq = 0    # mq = 0 : Height field
          # mq = 1 : Momentum field


#choice of the Riemann solver        
solver = 1 # 0 = Roe-solver
           # 1 = flux-decomposition solver
           # 2 = fwave Solver with source term

limiter_choice = 'minmod' #MC 
                          #minmod
                          #superbea
                          #vanleer
                
second_order = True

# Spatial domain
ax = -6
bx = 6

#middle dry state choose: ay = -0.1, by = 0.2
#left dry state choose: ay = -0.1, by = 1.2
#right dry state choose: ay = -0.1, by = 1.2
ay = -0.5    
by = 3

meqn = 2  # Number of equations in the system

# Gravity
g = 1

# Temporal domain
to = 0
Tfinal = 2

# Numerical parameters
cfl = 0.9

mx = 256 #number of spatial points

# Estimate maximum wave speed.  Note that this will change with time, so we have to find a 
# maximum that works for t in [0,T]
umax = 1.7

#### Sample test and plot

In [3]:
import test #Conatins test cases, plot function,Intial and Boundary conditions

test.Riemansoln(umax,to,mq,case,itype,g,ax,bx,ay,by,mx,\
                Tfinal,limiter_choice,second_order,meqn,\
                solver,cfl)

SystemExit: The approximate solves can't handle dry states yet, please choose itype = 0 for presence of dry states.